In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [2]:
erp_ptsku_clean = pd.read_csv('./Data_output/【已清洗】PTSKU_erp.csv', dtype=str).fillna('')
publish_sku_updata = pd.read_csv('./Data_output/【已更新】SKU_publish.csv', dtype=str).fillna('')
publish_sku_updata['itemid'] = publish_sku_updata['itemid'].astype(int)
publish_sku_updata['itemid'] = publish_sku_updata['itemid'].astype(str)
erp_ptsku_clean['sku_中性'] = erp_ptsku_clean['SKU'].copy()
erp_ptsku_clean['sku_中性'] = erp_ptsku_clean['sku_中性'].apply(lambda x:re.sub(r'[^a-zA-Z0-9]+[a-zA-Z0-9]*$', '', x))
used_columns = ['sku_匹配', 'country', 'site', 'selleruserid', 'itemid', 'sku', '清理后刊登oe']

In [3]:
# 左连接匹配
merge_data1 = pd.merge(erp_ptsku_clean[['SKU', 'sku_中性']], publish_sku_updata[used_columns], left_on='SKU', right_on='sku_匹配', how='left').fillna('')
merge_data2 = pd.merge(erp_ptsku_clean[['SKU', 'sku_中性']], publish_sku_updata[used_columns], left_on='sku_中性', right_on='sku_匹配', how='left').fillna('')

In [4]:
# 聚合
group_merge_data1 = merge_data1.groupby(['SKU'], as_index=False, sort=False).agg({'sku_匹配':list, 'country':list, 'site':list,
                                                                                'selleruserid':list, 'itemid':list, 'sku':list,
                                                                                '清理后刊登oe':list})
group_merge_data2 = merge_data2.groupby(['SKU'], as_index=False, sort=False).agg({'sku_匹配':list, 'country':list, 'site':list,
                                                                                'selleruserid':list, 'itemid':list, 'sku':list,
                                                                                '清理后刊登oe':list})

In [5]:
# 合并匹配数据
group_merge_data1['sku_匹配'] = group_merge_data1['sku_匹配'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data1['country'] = group_merge_data1['country'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data1['site'] = group_merge_data1['site'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data1['selleruserid'] = group_merge_data1['selleruserid'].apply(lambda x:';'.join(sorted(list(set([i for i in x if i != ''])), key=x.index)))
group_merge_data1['itemid'] = group_merge_data1['itemid'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data1['sku'] = group_merge_data1['sku'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data1['清理后刊登oe'] = group_merge_data1['清理后刊登oe'].apply(lambda x:';'.join(sorted(list(set([oe for oe in x if oe != ''])), key=x.index)))

group_merge_data2['sku_匹配'] = group_merge_data2['sku_匹配'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data2['country'] = group_merge_data2['country'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data2['site'] = group_merge_data2['site'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data2['selleruserid'] = group_merge_data2['selleruserid'].apply(lambda x:';'.join(sorted(list(set([i for i in x if i != ''])), key=x.index)))
group_merge_data2['itemid'] = group_merge_data2['itemid'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data2['sku'] = group_merge_data2['sku'].apply(lambda x:';'.join(sorted(list(set(x)), key=x.index)))
group_merge_data2['清理后刊登oe'] = group_merge_data2['清理后刊登oe'].apply(lambda x:';'.join(sorted(list(set([oe for oe in x if oe != ''])), key=x.index)))

In [6]:
# 连接两次匹配数据
final_merge_data = pd.merge(group_merge_data1, group_merge_data2, on='SKU', how='left').fillna('')
# 选取匹配数据，优先选择PTSKU匹配数据，之后选择中性SKU匹配数据
final_merge_data['sku_匹配'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['sku_匹配_x'], final_merge_data['sku_匹配_y'])
final_merge_data['country'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['country_x'], final_merge_data['country_y'])
final_merge_data['site'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['site_x'], final_merge_data['site_y'])
final_merge_data['selleruserid'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['selleruserid_x'], final_merge_data['selleruserid_y'])
final_merge_data['listing itemid'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['itemid_x'], final_merge_data['itemid_y'])
final_merge_data['listing sku'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['sku_x'], final_merge_data['sku_y'])
final_merge_data['刊登oe'] = np.where(final_merge_data['sku_匹配_x'] != '', final_merge_data['清理后刊登oe_x'], final_merge_data['清理后刊登oe_y'])

In [7]:
# 与erp数据合并更新最终查重用OE
final_merge_data_columns = ['SKU', 'sku_匹配', 'country', 'site', 'selleruserid', 'listing itemid', 'listing sku', '刊登oe']
final_result = pd.merge(erp_ptsku_clean[['SKU', 'OE']], final_merge_data[final_merge_data_columns], on='SKU', how='left').fillna('')
final_result['查重用OE'] = final_result[['OE','刊登oe']].apply(lambda x:';'.join(x),axis=1)
final_result['查重用OE'] = final_result['查重用OE'].apply(lambda x:';'.join([oe for oe in sorted(list(set(x.split(';'))), key=x.split(';').index) if oe != '']))
used_final_result = pd.merge(erp_ptsku_clean, final_result.drop(['OE'], axis=1), on='SKU', how='left').fillna('')
used_final_result

,SKU,GID,子市场,SKU类型,是否采购,商品层级,商品状态,供应商SKU,EAN,OE,...,商品分类(英文),sku_中性,sku_匹配,country,site,selleruserid,listing itemid,listing sku,刊登oe,查重用OE
0,IC10738-FC,IC10738,"FR,APM,DE,APM-VC,IT,UK,APM-UK",采购SKU,是,普通,正常,CR-IGN170B,,12137510738;12131487707,...,Ignition Coil,IC10738,IC10738,DE;GB;CA;FR;US;IT,Germany;UK;Canada;France;eBayMotors;Ireland;Sp...,apm-autoteile;premium-autoteile;carparts_premi...,263943945818.0;253883995842.0;202454719116.0;1...,DEIC10738;P-DEIC10738;Car-DEIC10738;AP-UKIC107...,12131487707;12137510738;APM10738;12131487707;1...,12137510738;12131487707;APM10738;PREMIUM10738;...
1,IC08063-FC,IC08063,"FR,AU,APM,DE,APM-VC,IT",采购SKU,是,普通,正常,O-IGN095,,1208063;10457075;1103905;1103872;DR44,...,Ignition Coil,IC08063,IC08063,DE;IT;FR,Belgium_French;Austria;France;Spain;Italy;Germany,premium-autoteile;carparts_premium;apm-autoteile,255045465306.0;255045465402.0;203607496698.0;2...,P-DEIC08063;Car-DEIC08063;DEIC08063;C-ITIC0806...,10457075;1103872;1103905;1103982;104003;120806...,1208063;10457075;1103905;1103872;DR44;1103982;...
2,WLA73401ZX,WLA73401,"APM,DE",采购SKU,是,普通,正常,KK25-5934,,1273401;1273403;1273420;93171541;93185524;9318...,...,Wiper Linkage Assembly,WLA73401ZX,,,,,,,,1273401;1273403;1273420;93171541;93185524;9318...
3,WIM20670ZX,WIM20670,"AP-CA,US-CA,US,AP,YHT",采购SKU,是,TOP20,重点维护,KK17-1867,,8C3Z17508;9L1417508;9C3417504;WM748;402067,...,Wiper Motor,WIM20670ZX,,,,,,,,8C3Z17508;9L1417508;9C3417504;WM748;402067
4,WIM02013-C1,WIM02013,"AP-CA,US-CA,US,AP",采购SKU,是,普通,正常,KK17-1834C,,6C2Z17508;F87Z17508;F8UU17504;ZZP067350;402013,...,Wiper Motor,WIM02013,WIM02013,US;CA,eBayMotors;Canada,autoparts_premium;a-premium-us;total_autopart;...,184350184112.0;124250458682.0;164282853713.0;2...,USWIM02013;AP-USWIM02013;T-USWIM02013;P-USWIM0...,1F8067350;1F8067350A;1F8067350R00;1F8067350R0A...,6C2Z17508;F87Z17508;F8UU17504;ZZP067350;402013...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153653,SGAU002,SGAU002,"AP-CA,US-CA,APM,DE,AP,US,YHT,FR,IT",采购SKU,是,普通,正常,AU2014,,20072015;Q7,...,Splash Guard,SGAU002,SGAU002,DE;US;CA,Germany;eBayMotors;Canada;Spain;France;Italy;B...,carparts_premium;a-premium-us;partswholesale2u...,203775379794.0;124246585736.0;283943631982.0;1...,Car-DESGAU002;AP-USSGAU002;P-USSGAU002;USSGAU0...,4L0075101A;4L0075111A;31216764443;0123K031;312...,20072015;Q7;4L0075101A;4L0075111A;31216764443;...
153654,SGAU001,SGAU001,"AP-CA,US-CA,APM,DE,AP,US,YHT,IT,FR",采购SKU,是,普通,正常,AU2013,,20072015;Q7,...,Splash Guard,SGAU001,SGAU001,DE;CA;US,Austria;Belgium_French;Italy;Spain;France;Irel...,premium-autoteile;carparts_premium;apm-autotei...,255045464174.0;255045464676.0;203643843225.0;2...,P-DESGAU001;Car-DESGAU001;DESGAU001;CASGAU001;...,AU001C;03RSA1007QBK;17217853240;8200146271;820...,20072015;Q7;AU001C;03RSA1007QBK;17217853240;82...
153655,SGAC005,SGAC005,"AP-CA,US-CA,AP,US,YHT",采购SKU,是,TOP20,正常,OC2002,,RDX20182019,...,Splash Guard,SGAC005,SGAC005,US;CA,eBayMotors;Canada,partswholesale2u;partswholesaleit;a-premium-us...,284169583988.0;313782254148.0;124256048378.0;1...,P-USSGAC005;P-USSGAC005-IT;AP-USSGAC005;USSGAC...,08P00TJB200,RDX20182019;08P00TJB200
153656,SGAC003,SGAC003,"US,YHT,AP",采购SKU,是,普通,正常,OC1001,,20132016,...,Splash Guard,SGAC003,SGAC003,US,eBayMotors,autoparts_premium;total_autopart;a-premium-us;...,172557592716.0;164276775973.0;124269191392.0;2...,USSGAC003;T-USSGAC003;AP-USSGAC003;P-USSGAC003,08P00TK4210B,20132016;08P00TK4210B


In [8]:
out_put_columns = ['SKU', '查重用OE', 'OE_original', '刊登oe', 'sku_中性', 'GID', '子市场', '品类简称',
                   '中文报关品名', '英文报关品名', '商品分类', '商品分类(中文)', '商品分类(英文)',
                   '商品层级', '供应商SKU', '商品描述', '最新价格（含税）',  '采购员', '供应商代码', '供应商简称',
                   '上下架', '创建日期', 'country', 'site', 'selleruserid', 'listing itemid', 'listing sku']
df_output = used_final_result[out_put_columns].copy()
df_output.rename(columns={'OE_original':'OE-ERP', '刊登oe':'OE-eBay'}, inplace=True)
df_output = df_output[df_output['供应商简称']!='PA'].copy()

In [9]:
# 保存输出
week_now = datetime.today().strftime('%W')
year_now = datetime.today().strftime('%Y')
output_path = './Data_output/' + year_now + '-W' + week_now + '-查重用OE.xlsx'
df_output.to_excel(output_path, index=False)